# 工具函数

In [16]:
import numpy as np

# 计算熵权的对数运算
def mylog(p):
    n = len(p)  # 获取输入向量p的长度
    lnp = np.zeros(n)  # 创建一个长度为n，元素都为0的新数组lnp
    for i in range(n):  # 对向量p的每一个元素进行循环
        if p[i] == 0:  # 如果当前元素的值为0
            lnp[i] = 0  # 则在lnp中对应位置也设置为0，因为log(0)是未定义的，这里我们规定为0
        else:
            lnp[i] = np.log(p[i])  # 如果p[i]不为0，则计算其自然对数并赋值给lnp的对应位置
    return lnp  # 返回计算后的对数数组

# 正向化函数
# 极小型指标转化为极大型指标的函数
def minTomax(maxx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    ans = [[(maxx-e)] for e in x]  # 计算最大值与每个指标值的差，并将其放入新列表中
    return np.array(ans)  # 将列表转换为numpy数组并返回

# 中间型指标转化为极大型指标的函数
def midTomax(bestx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    h = [abs(e-bestx) for e in x]  # 计算每个指标值与最优值之间的绝对差
    M = max(h)  # 找到最大的差值
    if M == 0:
        M = 1  # 防止最大差值为0的情况
    ans = [[(1-e/M)] for e in h]  # 计算每个差值占最大差值的比例，并从1中减去，得到新指标值
    return np.array(ans)  # 返回处理后的numpy数组

# 区间型指标转化为极大型指标的函数
def regTomax(lowx, highx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    M = max(lowx-min(x), max(x)-highx)  # 计算指标值超出区间的最大距离
    if M == 0:
        M = 1  # 防止最大距离为0的情况
    ans = []
    for i in range(len(x)):
        if x[i]<lowx:
            ans.append([(1-(lowx-x[i])/M)])  # 如果指标值小于下限，则计算其与下限的距离比例
        elif x[i]>highx:
            ans.append([(1-(x[i]-highx)/M)])  # 如果指标值大于上限，则计算其与上限的距离比例
        else:
            ans.append([1])  # 如果指标值在区间内，则直接取为1
    return np.array(ans)  # 返回处理后的numpy数组



# Step1. 数据标准化流程

In [17]:
A = [[9,10,175,120], [8,7,164,70], [6,3,157,90]]
A = np.array(A,dtype=float)
objectNum = len(A)
indicatorNum = len(A[0])

# 指标类型，按顺序输入，1:极大型，2：极小型，3：中间型，4：区间型
kind = [1,2,3,4]

# 中间值的最优值
bestA = 165
# 区间型的最大值和最小值
lowA = 90
highA = 100

print(A)

[[  9.  10. 175. 120.]
 [  8.   7. 164.  70.]
 [  6.   3. 157.  90.]]


## 数据正向化

In [18]:
X = np.zeros(shape=(objectNum, 0))  # 初始化为 0 列，因为后续会逐列拼接

for i in range(indicatorNum):
    if kind[i] == 1:  # 如果当前指标为极大型，则直接使用原值
        v = np.array(A[:, i])
    elif kind[i] == 2:  # 如果当前指标为极小型，调用 minTomax 函数转换
        maxA = max(A[:, i])
        v = minTomax(maxA, A[:, i])
    elif kind[i] == 3:  # 如果当前指标为中间型，调用 midTomax 函数转换
        v = midTomax(bestA, A[:, i])
    elif kind[i] == 4:  # 如果当前指标为区间型，调用 regTomax 函数转换
        v = regTomax(lowA, highA, A[:, i])
    
    if i==0:
        X = v.reshape(-1, 1)
    else:
        X = np.hstack([X, v])  # 否则，将新指标列拼接到 X 数组上

print("统一指标后矩阵为：\n{}".format(X))  # 打印处理后的矩阵 X

统一指标后矩阵为：
[[9.  0.  0.  0. ]
 [8.  3.  0.9 0. ]
 [6.  7.  0.2 1. ]]


## 数据标准化
- 去量纲

In [19]:

Z = X / np.sqrt(np.sum(X*X, axis=0))
Z_abs = abs(Z) 

print(Z_abs)

[[0.66896473 0.         0.         0.        ]
 [0.59463532 0.3939193  0.97618706 0.        ]
 [0.44597649 0.91914503 0.21693046 1.        ]]


## 数据归一化

In [20]:
ASum = np.sum(Z_abs, axis=0)
print('ASum=', ASum)
# 归一化后的矩阵
Stand_A = Z_abs / ASum

print(Stand_A)

ASum= [1.70957654 1.31306433 1.19311752 1.        ]
[[0.39130435 0.         0.         0.        ]
 [0.34782609 0.3        0.81818182 0.        ]
 [0.26086957 0.7        0.18181818 1.        ]]


# Step2. 计算熵权

In [21]:
D = np.zeros(indicatorNum)  # 初始化一个长度为m的数组D，用于保存每个指标的信息效用值
# 计算每个指标的信息效用值
for i in range(indicatorNum):  # 遍历Z的每一列
    x = Stand_A[:, i]  # 获取Z的第i列，即第i个指标的所有数据
    p = x / np.sum(x)  # 对第i个指标的数据进行归一化处理，得到概率分布p
    # 使用自定义的mylog函数计算p的对数。需要注意的是，如果p中含有0，直接使用np.log会得到-inf，这里使用自定义函数避免这个问题
    e = -np.sum(p * mylog(p)) / np.log(objectNum)  # 根据熵的定义计算第i个指标的信息熵e
    D[i] = 1 - e  # 根据信息效用值的定义计算D[i]

# 根据信息效用值计算各指标的权重
W = D / np.sum(D)  # 将信息效用值D归一化，得到各指标的权重W

# 取三位数
print("权重 weights_abs = ")
print(W)  # 打印得到的权重数组W

权重 weights_abs = 
[0.00611442 0.21926832 0.28073338 0.49388388]


# Step3. 计算结果

In [22]:
marks = [0] * objectNum
for i in range(objectNum):
    sum = 0
    for j in range(indicatorNum):
        sum += Stand_A[i][j] * W[j]
    marks[i] =sum

print(marks)

[0.0023926000338037167, 0.29759819581448405, 0.7000092041517122]
